In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

from matplotlib import cm, colors
import matplotlib.pyplot as plt

import numpy as np

import dataUtils as du

import tempfile
import os

def GetData(filename, train_frac=0.5, branches=['pileup', 'amp', 'phase', 'trace'],  treename="OutputTree"):
    '''
    Returns TFile as a pandas dataframe
    '''
    try: 
        import uproot
        import awkward as ak
        import numpy as np
        from sklearn.model_selection import train_test_split
        
    except ImportError:
        print("Please install uproot, awkward, numpy, and pandas")
        return
  
    file = uproot.open(filename)
    tree = file[treename]
    data = {}
    for branch in branches:
        data[branch] = ak.to_numpy(tree[branch].arrays()[branch])
    
    train_size = int(len(data["trace"])*train_frac)
    test_size = len(data["trace"]) - train_size
    print("Train size: ",train_size)
    print("Test size: ",test_size)

    x = np.array(data["trace"])
    y_pileup = np.array(data["pileup"])
    y_amp = np.array(data["amp"])
    y_phase = np.array(data["phase"])

    x_train, x_test, y_pileup_train, y_pileup_test, y_amp_train, y_amp_test, y_phase_train, y_phase_test = train_test_split(x, y_pileup, y_amp, y_phase, test_size=test_size)

    return ((x_train, y_pileup_train, y_amp_train, y_phase_train), (x_test, y_pileup_test, y_amp_test, y_phase_test))

(x_train, y_pileup_train, y_amp_train, y_phase_train), (x_test, y_pileup_test, y_amp_test, y_phase_test) = GetData("../data/Data10percFloat.root")

2024-03-27 19:14:22.538512: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 19:14:22.540173: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-27 19:14:22.567392: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 19:14:22.567418: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 19:14:22.567434: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Train size:  298980
Test size:  298981


In [ ]:
def PileupNet(inputSize, 
                kernel_size = 4,
                filters = 16,
                dense_units = 32,
                verbose=True,
                kernel_reg = False,
                lambda_reg = 0.000
                ):


    if kernel_reg:
         model = keras.Sequential(
              [
                    tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='relu', input_shape=(inputSize,1), kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='conv1'),
                    tf.keras.layers.Flatten(name='flatten'),
                    tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='dense'),
                    tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='output')
                ],
                name='PileupNet'
            )
    else:
        model = keras.Sequential(
            [
                tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='relu', input_shape=(inputSize,1), kernel_initializer='he_uniform', name='conv1'),
                tf.keras.layers.Flatten(name='flatten'),
                tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', name='dense'),
                tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_uniform', name='output')
            ],
            name='PileupNet'
        )

    if verbose:
        model.summary()
    return model

def AmpNet(inputSize, 
                kernel_size = 10,
                filters = 16,
                dense_units = 128,
                verbose=True,
                kernel_reg = False,
                lambda_reg = 0.000
                ):


    if kernel_reg:
         model = keras.Sequential(
              [
                    tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='tanh', input_shape=(inputSize,1), kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='conv1'),
                    tf.keras.layers.Flatten(name='flatten'),
                    tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='dense'),
                    tf.keras.layers.Dense(1, activation='linear', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='output')
                ],
                name='AmpNet'
            )
    else:
        model = keras.Sequential(
            [
                tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='tanh', input_shape=(inputSize,1), kernel_initializer='he_uniform', name='conv1'),
                tf.keras.layers.Flatten(name='flatten'),
                tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', name='dense'),
                tf.keras.layers.Dense(1, activation='linear', kernel_initializer='he_uniform', name='output')
            ],
            name='AmpNet'
        )

    if verbose:
        model.summary()
    return model

def PhaseNet(inputSize, 
                kernel_size = 10,
                filters = 16,
                dense_units = 128,
                verbose=True,
                kernel_reg = False,
                lambda_reg = 0.000
            ):


    if kernel_reg:
         model = keras.Sequential(
              [
                    tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='relu', input_shape=(inputSize,1), kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='conv1'),
                    tf.keras.layers.Flatten(name='flatten'),
                    tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='dense'),
                    tf.keras.layers.Dense(1, activation='linear', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='output')
                ],
                name='PhaseNet'
            )
    else:
        model = keras.Sequential(
            [
                tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='relu', input_shape=(inputSize,1), kernel_initializer='he_uniform', name='conv1'),
                tf.keras.layers.Flatten(name='flatten'),
                tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', name='dense'),
                tf.keras.layers.Dense(1, activation='linear', kernel_initializer='he_uniform', name='output')
            ],
            name='PhaseNet'
        )

    if verbose:
        model.summary()
    return model

In [ ]:

'''
Model definitions for the network pruning project
'''
import sys

try: 
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    import tensorflow_model_optimization as tfmot
    from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude
except ImportError:
    print("Please install tensorflow, numpy, and pandas")
    sys.exit(1)

def PhaseNet(inputsize):
    model = keras.Sequential([
        layers.Conv1D(kernel_size=10, filters=64, activation='tanh', name='conv1', input_shape=(inputsize,1)),
        layers.Dropout(0.2, name='dropout1'),
        layers.Conv1D(kernel_size=1, filters=64, activation='relu', name='conv2'),
        layers.Dropout(0.2, name='dropout2'),
        layers.Conv1D(kernel_size=2, filters=64, activation='relu', padding='same', name='conv3'),
        layers.Dropout(0.2, name='dropout3'),
        layers.Flatten(name='flat1'),
        layers.Dense(128, activation='relu', name='dense1'),
        layers.Dense(1, activation='linear', name='output')
    ],
    name='PhaseNet')
    return model

def PhaseNetPrunable(inputsize, pruning_params_2_by_4, pruning_params_sparsity_0_5):
    model = keras.Sequential([
        prune_low_magnitude(
            layers.Conv1D(kernel_size=10, filters=64, activation='tanh', name='conv1', input_shape=(inputsize,1)),
            **pruning_params_sparsity_0_5),
        layers.Dropout(0.2, name='dropout1'),
        prune_low_magnitude(
            layers.Conv1D(kernel_size=1, filters=64, activation='relu', name='conv2'),
            **pruning_params_sparsity_0_5),
        layers.Dropout(0.2, name='dropout2'),
        prune_low_magnitude(
            layers.Conv1D(kernel_size=2, filters=64, activation='relu', padding='same', name='conv3'),
            **pruning_params_2_by_4),
        layers.Dropout(0.2, name='dropout3'),
        layers.Flatten(name='flat1'),
        prune_low_magnitude(
            layers.Dense(128, activation='relu', name='dense1'),
            **pruning_params_2_by_4),
        prune_low_magnitude(
            layers.Dense(1, activation='linear', name='output'),
            **pruning_params_sparsity_0_5)
    ], name='PhaseNetPrunable')

    return model